In [2]:
import pandas as pd

In [3]:
import numpy as np

class ObjectDetectionMetrics:
    def __init__(self):
        self.classes_metrics = []

    def add_class_metrics(self, true_positive, false_positive, false_negative):
        metrics = {
            'TP': np.array([true_positive]),
            'FP': np.array([false_positive]),
            'FN': np.array([false_negative])
        }
        self.classes_metrics.append(metrics)

    def calculate_precision_recall(self, metrics):
        TP = metrics['TP']
        FP = metrics['FP']
        FN = metrics['FN']
        precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        return precision, recall

    def average_precision(self, precision, recall):
        # En este contexto simplificado, podemos considerar que la precisión y el recall son constantes
        return precision[0]  # Devuelve el único valor de precisión

    def calculate_mAP(self):
        aps = []
        for metrics in self.classes_metrics:
            precision, recall = self.calculate_precision_recall(metrics)
            ap = self.average_precision(precision, recall)
            aps.append(ap)
        return np.mean(aps)

# Ejemplo de uso
od_metrics = ObjectDetectionMetrics()
od_metrics.add_class_metrics(100, 10, 5) # TP, FP, FN
od_metrics.add_class_metrics(90, 20, 10)

# Calculando precisión, recall y AP para cada clase
for i, metrics in enumerate(od_metrics.classes_metrics):
    precision, recall = od_metrics.calculate_precision_recall(metrics)
    ap = od_metrics.average_precision(precision, recall)
    print(f"Clase {i+1} - Precisión: {precision[0]:.2f}, Recall: {recall[0]:.2f}, AP: {ap:.2f}")

# Calculando mAP para el modelo
mAP = od_metrics.calculate_mAP()
print(f"Mean Average Precision (mAP) del modelo: {mAP:.2f}")

Clase 1 - Precisión: 0.91, Recall: 0.95, AP: 0.91
Clase 2 - Precisión: 0.82, Recall: 0.90, AP: 0.82
Mean Average Precision (mAP) del modelo: 0.86


# FP32

In [4]:
file_path = "results/model-validation/FP32.csv"
df32 = pd.read_csv(file_path)
df32.head()

,image,gt_x1,gt_y1,gt_x2,gt_y2,pr_x1,pr_y1,pr_x2,pr_y2,pr_score,IoU
0,img_20230322_224503.jpg,3659,2635,3777,2990,3661.0,2638.0,3809.0,2901.0,0.776861,0.606449
1,img_20230708_223004.jpg,2751,2609,3111,2808,2746.0,2611.0,3036.0,2806.0,0.876259,0.765338
2,img_20230624_014503.jpg,3121,731,3299,1048,3113.0,730.0,3307.0,1059.0,0.902837,0.884060
3,img_20230718_001503.jpg,3366,2088,3643,2354,3370.0,2097.0,3622.0,2356.0,0.862179,0.872995
4,img_20230616_224503.jpg,1288,2412,1539,2606,1287.0,2418.0,1540.0,2613.0,0.893374,0.928149


In [5]:
# Contar las filas que contienen al menos un NaN
num_rows_with_nan = df32.isna().any(axis=1).sum()
print(f"Número de filas con al menos un valor NaN: {num_rows_with_nan} de {df32.shape[0]}")
# Calcular el porcentaje de filas donde hay NaN
percentage = (num_rows_with_nan / df32.shape[0]) * 100
print(f"Porcentaje de filas con al menos un valor NaN: {percentage:.2f}%")

Número de filas con al menos un valor NaN: 2 de 147
Porcentaje de filas con al menos un valor NaN: 1.36%


In [6]:
# Contar las filas donde la columna 'IoU' es mayor o igual a 0.5
num_rows_iou_ge_0_5 = (df32['IoU'] >= 0.5).sum()
print(f"Número de filas donde 'IoU' es mayor o igual a 0.5: {num_rows_iou_ge_0_5} de {df32.shape[0]}")


# Calcular el porcentaje de filas donde 'IoU' es mayor o igual a 0.5
percentage = (num_rows_iou_ge_0_5 / df32.shape[0]) * 100
print(f"Porcentaje de filas donde 'IoU' es mayor o igual a 0.5: {percentage:.2f}%")

# Calcular el valor promedio de la columna 'IoU'
mean_iou = df32['IoU'].mean()
print(f"El valor promedio de IoU es: {mean_iou:.2f}")

# Calcular la desviación estándar de la columna 'IoU'
std_iou = df32['IoU'].std()
print(f"La desviación estándar de IoU es: {std_iou:.2f}")

Número de filas donde 'IoU' es mayor o igual a 0.5: 141 de 147
Porcentaje de filas donde 'IoU' es mayor o igual a 0.5: 95.92%
El valor promedio de IoU es: 0.83
La desviación estándar de IoU es: 0.15


In [7]:
TP = num_rows_iou_ge_0_5
FP = 0 # Porqué todos son superiores a 0.5
FN = num_rows_with_nan

od_metrics = ObjectDetectionMetrics()
od_metrics.add_class_metrics(TP, FP, FN)

# Calculando precisión, recall y AP para cada clase
for i, metrics in enumerate(od_metrics.classes_metrics):
    precision, recall = od_metrics.calculate_precision_recall(metrics)
    ap = od_metrics.average_precision(precision, recall)
    print(f"Clase {i+1} - Precisión: {precision[0]:.2f}, Recall: {recall[0]:.2f}, AP: {ap:.2f}")

# Calculando mAP para el modelo
mAP = od_metrics.calculate_mAP()
print(f"Mean Average Precision (mAP) del modelo: {mAP:.2f}")

Clase 1 - Precisión: 1.00, Recall: 0.99, AP: 1.00
Mean Average Precision (mAP) del modelo: 1.00


Hasta ahora es el paso a paso y tendriamos el mAP50. Hagamos el bucle para obtener el map50-95.

In [8]:
mAPs = []
for iou in range(50, 100, 5):
    TP = (df32['IoU'] >= iou/100).sum()
    FP = (df32['IoU'] < iou/100).sum()
    FN = df32.isna().any(axis=1).sum()

    od_metrics = ObjectDetectionMetrics()
    od_metrics.add_class_metrics(TP, FP, FN)

    # Calculando precisión, recall y AP para cada clase
    for i, metrics in enumerate(od_metrics.classes_metrics):
        precision, recall = od_metrics.calculate_precision_recall(metrics)
        ap = od_metrics.average_precision(precision, recall)
        # print(f"IoU {iou/100} - Precisión: {precision[0]:.2f}, Recall: {recall[0]:.2f}, AP: {ap:.2f}")

    # Calculando mAP para el modelo
    mAP = od_metrics.calculate_mAP()
    print(f"  ·IoU {iou/100} -  Mean Average Precision (mAP) del modelo: {mAP*100:.2f}")

    mAPs.append(mAP)

print(f'\nFINAL mAP50-95: {np.mean(mAPs)*100:.2f}')

  ·IoU 0.5 -  Mean Average Precision (mAP) del modelo: 97.24
  ·IoU 0.55 -  Mean Average Precision (mAP) del modelo: 97.24
  ·IoU 0.6 -  Mean Average Precision (mAP) del modelo: 96.55
  ·IoU 0.65 -  Mean Average Precision (mAP) del modelo: 94.48
  ·IoU 0.7 -  Mean Average Precision (mAP) del modelo: 89.66
  ·IoU 0.75 -  Mean Average Precision (mAP) del modelo: 86.90
  ·IoU 0.8 -  Mean Average Precision (mAP) del modelo: 77.24
  ·IoU 0.85 -  Mean Average Precision (mAP) del modelo: 59.31
  ·IoU 0.9 -  Mean Average Precision (mAP) del modelo: 25.52
  ·IoU 0.95 -  Mean Average Precision (mAP) del modelo: 2.07

FINAL mAP50-95: 72.62


# FP16

In [9]:
file_path = "results/model-validation/FP16.csv"
df16 = pd.read_csv(file_path)
df16.head()

,image,gt_x1,gt_y1,gt_x2,gt_y2,pr_x1,pr_y1,pr_x2,pr_y2,pr_score,IoU
0,img_20230322_224503.jpg,3659,2635,3777,2990,3662.0,2639.0,3810.0,2902.0,0.776977,0.598094
1,img_20230708_223004.jpg,2751,2609,3111,2808,2747.0,2612.0,3037.0,2807.0,0.876298,0.770091
2,img_20230624_014503.jpg,3121,731,3299,1048,3114.0,731.0,3308.0,1060.0,0.902805,0.884060
3,img_20230718_001503.jpg,3366,2088,3643,2354,3371.0,2097.0,3623.0,2356.0,0.862200,0.872995
4,img_20230616_224503.jpg,1288,2412,1539,2606,1287.0,2418.0,1539.0,2613.0,0.893417,0.931722


In [10]:
# Contar las filas que contienen al menos un NaN
num_rows_with_nan = df16.isna().any(axis=1).sum()
print(f"Número de filas con al menos un valor NaN: {num_rows_with_nan} de {df16.shape[0]}")
# Calcular el porcentaje de filas donde hay NaN
percentage = (num_rows_with_nan / df16.shape[0]) * 100
print(f"Porcentaje de filas con al menos un valor NaN: {percentage:.2f}%")

Número de filas con al menos un valor NaN: 2 de 147
Porcentaje de filas con al menos un valor NaN: 1.36%


In [11]:
# Contar las filas donde la columna 'IoU' es mayor o igual a 0.5
num_rows_iou_ge_0_5 = (df16['IoU'] >= 0.5).sum()
print(f"Número de filas donde 'IoU' es mayor o igual a 0.5: {num_rows_iou_ge_0_5} de {df16.shape[0]}")


# Calcular el porcentaje de filas donde 'IoU' es mayor o igual a 0.5
percentage = (num_rows_iou_ge_0_5 / df16.shape[0]) * 100
print(f"Porcentaje de filas donde 'IoU' es mayor o igual a 0.5: {percentage:.2f}%")

# Calcular el valor promedio de la columna 'IoU'
mean_iou = df16['IoU'].mean()
print(f"El valor promedio de IoU es: {mean_iou:.2f}")

# Calcular la desviación estándar de la columna 'IoU'
std_iou = df16['IoU'].std()
print(f"La desviación estándar de IoU es: {std_iou:.2f}")

Número de filas donde 'IoU' es mayor o igual a 0.5: 141 de 147
Porcentaje de filas donde 'IoU' es mayor o igual a 0.5: 95.92%
El valor promedio de IoU es: 0.83
La desviación estándar de IoU es: 0.15


In [12]:
mAPs = []
for iou in range(50, 100, 5):
    TP = (df16['IoU'] >= iou/100).sum()
    FP = (df16['IoU'] < iou/100).sum()
    FN = df16.isna().any(axis=1).sum()

    od_metrics = ObjectDetectionMetrics()
    od_metrics.add_class_metrics(TP, FP, FN)

    # Calculando precisión, recall y AP para cada clase
    for i, metrics in enumerate(od_metrics.classes_metrics):
        precision, recall = od_metrics.calculate_precision_recall(metrics)
        ap = od_metrics.average_precision(precision, recall)
        # print(f"IoU {iou/100} - Precisión: {precision[0]:.2f}, Recall: {recall[0]:.2f}, AP: {ap:.2f}")

    # Calculando mAP para el modelo
    mAP = od_metrics.calculate_mAP()
    print(f"  · IoU {iou/100} - Mean Average Precision (mAP) del modelo: {mAP*100:.2f}")

    mAPs.append(mAP)

print(f'\nFINAL mAP50-95: {np.mean(mAPs)*100:.2f}')

  · IoU 0.5 - Mean Average Precision (mAP) del modelo: 97.24
  · IoU 0.55 - Mean Average Precision (mAP) del modelo: 96.55
  · IoU 0.6 - Mean Average Precision (mAP) del modelo: 95.86
  · IoU 0.65 - Mean Average Precision (mAP) del modelo: 93.79
  · IoU 0.7 - Mean Average Precision (mAP) del modelo: 89.66
  · IoU 0.75 - Mean Average Precision (mAP) del modelo: 86.21
  · IoU 0.8 - Mean Average Precision (mAP) del modelo: 78.62
  · IoU 0.85 - Mean Average Precision (mAP) del modelo: 60.00
  · IoU 0.9 - Mean Average Precision (mAP) del modelo: 27.59
  · IoU 0.95 - Mean Average Precision (mAP) del modelo: 2.07

FINAL mAP50-95: 72.76


# INT8

In [18]:
file_path = "results/model-validation/INT8.csv"
df8 = pd.read_csv(file_path)
df8.head()

,image,gt_x1,gt_y1,gt_x2,gt_y2,pr_x1,pr_y1,pr_x2,pr_y2,pr_score,IoU
0,img_20230322_224503.jpg,3659,2635,3777,2990,1415,3256,1761,3461,0.988235,0.0
1,img_20230322_224503.jpg,3659,2635,3777,2990,1671,3256,2090,3461,0.988235,0.0
2,img_20230322_224503.jpg,3659,2635,3777,2990,667,645,904,1056,0.980392,0.0
3,img_20230322_224503.jpg,3659,2635,3777,2990,1223,3276,1442,3440,0.972549,0.0
4,img_20230322_224503.jpg,3659,2635,3777,2990,3633,754,3925,1055,0.945098,0.0


In [19]:
# Contar las filas que contienen al menos un NaN
num_rows_with_nan = df8.isna().any(axis=1).sum()
print(f"Número de filas con al menos un valor NaN: {num_rows_with_nan} de {df8.shape[0]}")
# Calcular el porcentaje de filas donde hay NaN
percentage = (num_rows_with_nan / df8.shape[0]) * 100
print(f"Porcentaje de filas con al menos un valor NaN: {percentage:.2f}%")

Número de filas con al menos un valor NaN: 0 de 3846
Porcentaje de filas con al menos un valor NaN: 0.00%


In [20]:
# Contar las filas donde la columna 'IoU' es mayor o igual a 0.5
num_rows_iou_ge_0_5 = (df8['IoU'] >= 0.5).sum()
print(f"Número de filas donde 'IoU' es mayor o igual a 0.5: {num_rows_iou_ge_0_5} de {df16.shape[0]}")


# Calcular el porcentaje de filas donde 'IoU' es mayor o igual a 0.5
percentage = (num_rows_iou_ge_0_5 / df8.shape[0]) * 100
print(f"Porcentaje de filas donde 'IoU' es mayor o igual a 0.5: {percentage:.2f}%")

# Calcular el valor promedio de la columna 'IoU'
mean_iou = df8['IoU'].mean()
print(f"El valor promedio de IoU es: {mean_iou:.2f}")

# Calcular la desviación estándar de la columna 'IoU'
std_iou = df8['IoU'].std()
print(f"La desviación estándar de IoU es: {std_iou:.2f}")

Número de filas donde 'IoU' es mayor o igual a 0.5: 5 de 147
Porcentaje de filas donde 'IoU' es mayor o igual a 0.5: 0.13%
El valor promedio de IoU es: 0.00
La desviación estándar de IoU es: 0.03


In [22]:
mAPs = []
for iou in range(50, 100, 5):
    TP = (df8['IoU'] >= iou/100).sum()
    FP = (df8['IoU'] < iou/100).sum()
    FN = df8.isna().any(axis=1).sum()

    od_metrics = ObjectDetectionMetrics()
    od_metrics.add_class_metrics(TP, FP, FN)

    # Calculando precisión, recall y AP para cada clase
    for i, metrics in enumerate(od_metrics.classes_metrics):
        precision, recall = od_metrics.calculate_precision_recall(metrics)
        ap = od_metrics.average_precision(precision, recall)
        # print(f"IoU {iou/100} - Precisión: {precision[0]:.2f}, Recall: {recall[0]:.2f}, AP: {ap:.2f}")

    # Calculando mAP para el modelo
    mAP = od_metrics.calculate_mAP()
    print(f"  · IoU {iou/100} - Mean Average Precision (mAP) del modelo: {mAP*100:.2f}")

    mAPs.append(mAP)

print(f'\nFINAL mAP50-95: {np.mean(mAPs)*100:.2f}')

  · IoU 0.5 - Mean Average Precision (mAP) del modelo: 0.13
  · IoU 0.55 - Mean Average Precision (mAP) del modelo: 0.05
  · IoU 0.6 - Mean Average Precision (mAP) del modelo: 0.03
  · IoU 0.65 - Mean Average Precision (mAP) del modelo: 0.03
  · IoU 0.7 - Mean Average Precision (mAP) del modelo: 0.00
  · IoU 0.75 - Mean Average Precision (mAP) del modelo: 0.00
  · IoU 0.8 - Mean Average Precision (mAP) del modelo: 0.00
  · IoU 0.85 - Mean Average Precision (mAP) del modelo: 0.00
  · IoU 0.9 - Mean Average Precision (mAP) del modelo: 0.00
  · IoU 0.95 - Mean Average Precision (mAP) del modelo: 0.00

FINAL mAP50-95: 0.02


/tmp/ipykernel_1509088/2122964467.py:20: RuntimeWarning: invalid value encountered in divide
  recall = TP / (TP + FN)
